<a href="https://colab.research.google.com/github/causallycausal/fastai/blob/dev/%20fastai/Projects/02_pizza_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook
import fastbook 
fastbook.setup_book()

Mounted at /content/gdrive


In [2]:
from fastbook import *
from fastai.vision.widgets import *

In [4]:
# Search key using duck duck go search engine.
# current fastai version is broken 
def search_images_ddg(key,max_n=200):
     """Search for 'key' with DuckDuckGo and return a unique urls of 'max_n' images
        (Adopted from https://github.com/deepanprabhu/duckduckgo-images-api)
     """
     url        = 'https://duckduckgo.com/'
     params     = {'q':key}
     res        = requests.post(url,data=params)
     searchObj  = re.search(r'vqd=([\d-]+)\&',res.text)
     if not searchObj: print('Token Parsing Failed !'); return
     requestUrl = url + 'i.js'
     headers    = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:71.0) Gecko/20100101 Firefox/71.0'}
     params     = (('l','us-en'),('o','json'),('q',key),('vqd',searchObj.group(1)),('f',',,,'),('p','1'),('v7exp','a'))
     urls       = []
     while True:
         try:
             res  = requests.get(requestUrl,headers=headers,params=params)
             data = json.loads(res.text)
             for obj in data['results']:
                 urls.append(obj['image'])
                 max_n = max_n - 1
                 if max_n < 1: return L(set(urls))     # dedupe
             if 'next' not in data: return L(set(urls))
             requestUrl = url + data['next']
         except:
             pass

# Goal 

Our goal with this project is to design an application that will classify three different types of pizza. The three categories will be cheese, pepperoni, and Hawaiian (ham and pineapple) pizza. In this notebook, we will gather the data, build the model, and  export the weights of the trained model to be used in a fun webapp. I will link to the webapp here after I have completed this project. 

# Gather Images of Pizza



In [3]:
pizza_types = ["cheese", "pepperoni", "Hawaiian"]
path = Path('pizzas')

In [11]:
if not path.exists():
  path.mkdir()
  for pizza in pizza_types: 
    dest = (path/pizza)
    dest.mkdir(exist_ok = True)
    results = search_images_ddg(f'{pizza} pizza')
    download_images(dest, urls=results)

In [12]:
image_paths = get_image_files(path)
image_paths

(#521) [Path('pizzas/Hawaiian/00000093.jpg'),Path('pizzas/Hawaiian/00000111.jpg'),Path('pizzas/Hawaiian/00000021.png'),Path('pizzas/Hawaiian/00000095.jpg'),Path('pizzas/Hawaiian/00000009.jpg'),Path('pizzas/Hawaiian/00000057.jpg'),Path('pizzas/Hawaiian/00000166.jpg'),Path('pizzas/Hawaiian/00000113.jpg'),Path('pizzas/Hawaiian/00000023.jpg'),Path('pizzas/Hawaiian/00000149.jpg')...]

# Verify our Images 

In [14]:
# Check if we can open our images 
failed = verify_images(image_paths)
failed

(#3) [Path('pizzas/Hawaiian/00000034.jpg'),Path('pizzas/cheese/00000104.png'),Path('pizzas/cheese/00000070.jpeg')]

In [ ]:
# Remove all images that we can't open 
failed.map(Path.unlink);

# Set up DataLoader 

In [ ]:
pizzas = Data